In [1]:
library(reticulate)
use_condaenv("dors")

In [2]:
library(DORS);

In [3]:
db <- dbConnect(DORS::DORS(), CTX = 'distributed')

In [4]:
db$dors_client

<Client: 'tcp://127.0.0.1:49886' processes=4 threads=12, memory=17.18 GB>

In [5]:
db$dors_client$dashboard_link

http://127.0.0.1:8787/status

In [6]:
dbWriteTable(db, "mtcars", mtcars)

In [7]:
names(db$tables)

[1] "mtcars"

In [8]:
res <- dbGetQuery(db, "SELECT * FROM mtcars WHERE cyl > 4")

In [9]:
head(res)

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AMC Javelin,15.2,8,304,150,3.15,3.435,17.30,0,0,3,2
Cadillac Fleetwood,10.4,8,472,205,2.93,5.250,17.98,0,0,3,4
Camaro Z28,13.3,8,350,245,3.73,3.840,15.41,0,0,3,4
Chrysler Imperial,14.7,8,440,230,3.23,5.345,17.42,0,0,3,4
Dodge Challenger,15.5,8,318,150,2.76,3.520,16.87,0,0,3,2
Duster 360,14.3,8,360,245,3.21,3.570,15.84,0,0,3,4


In [10]:
dbWriteTable(db, "taxi", "./sample_taxi.parquet")

In [11]:
dbGetQuery(db, "select count(*) from taxi")

COUNT(*)
<dbl>
743660


In [13]:
res <- dbGetQuery(db, "select avg(trip_distance), passenger_count 
                       from taxi 
                       group by passenger_count")

In [14]:
res

"AVG(""taxi"".""trip_distance"")",passenger_count
<dbl>,<dbl>
1.845263,0
2.484745,1
2.344380,2
2.313976,3
2.346265,4
2.325405,5
2.273989,6
0.000000,9


In [15]:
create_timeseries(db)
create_timeseries(db, 'ts2')
db$tables

{'mtcars': <dask_sql.datacontainer.DataContainer>, 'taxi': <dask_sql.datacontainer.DataContainer>, 'ts': <dask_sql.datacontainer.DataContainer>, 'ts2': <dask_sql.datacontainer.DataContainer>}

In [16]:
res <- dbGetQuery(db, "select * from ts limit 5")
res

,id,name,x,y
,<dbl>,<chr>,<dbl>,<dbl>
2000-01-01 00:00:00,1009,Ray,0.05501582,-0.08005533
2000-01-01 00:10:00,966,Sarah,0.91420733,0.27484151
2000-01-01 00:20:00,1014,Ursula,-0.09410239,0.55545577
2000-01-01 00:30:00,975,Zelda,0.72316210,0.24852102
2000-01-01 00:40:00,1012,Charlie,-0.37246921,0.28476695


In [17]:
#df.groupby('name').x.std()
res <- dbGetQuery(db, "select avg(x),name 
                       from ts 
                       group by name")

In [18]:
# df.join(df2, on=['id'])
res <- dbGetQuery(db, "select lhs.id, lhs.name, rhs.name 
                       from ts as lhs
                       INNER JOIN ts2 as rhs
                       ON
                       lhs.id = rhs.id")

Warning message in py_to_r.pandas.core.frame.DataFrame(dfFetch):
“index contains duplicated values: row names not set”


In [22]:
head(res)

,id,name,name0
,<dbl>,<chr>,<chr>
1,1044,Oliver,Dan
2,1044,Oliver,Zelda
3,1044,Oliver,George
4,1044,Oliver,Ray
5,1044,Oliver,Ursula
6,1044,Oliver,Bob
